# Recap of Large Language Models

Learn the basics

## Recap LLMs ~ Generate an answer to a single question

In [1]:
from dotenv import load_dotenv
import logging

from bring_a_crew.setup_logging import setup_logging
from ollama import chat, generate


In [2]:
MODEL = "phi4"

_ = load_dotenv()
setup_logging()
main_log = logging.getLogger("main")
main_log.setLevel(logging.INFO)


In [3]:
def run_question(question):
    llm_response = generate(
        prompt=question,
        model=MODEL,
        options={
            "temperature": 1
        }
    )
    main_log.debug(llm_response)
    return llm_response.response


In [4]:
print(run_question("When is Jettro available?"))

Jettro, a character from the TV series "Huntik: Secrets & Seekers," doesn't have specific availability times like a game or service might. In the context of the show, his presence and involvement depend on the storyline.

If you're looking to watch episodes featuring Jettro:

1. **Streaming Services:** Check platforms that offer "Huntik" to find available episodes.
2. **DVD Collections:** Purchase or rent physical copies if they are available in your region.
3. **Broadcast Schedule:** Look up TV listings for channels airing the series, if applicable.

For other contexts, like video games or merchandise related to Jettro, you might want to check specific platforms or retailers for availability details.


# Recap LLMs ~ System message

With a system message, you tell the LLM more about the role

In [5]:
print(run_question("I need to make an appointment with Jettro, when is he available?"))

As a large language model, I don't have access to specific personal schedules or real-time information. To find out when Jettro is available for an appointment, consider the following steps:

1. **Contact Directly**: Reach out to him directly via email, phone, or messaging platform if you have his contact details.

2. **Check Online Calendars**: If he uses a shared online calendar (like Google Calendar) and has granted you access, you can check for available slots there.

3. **Assistant or Office**: Contact his assistant or the office through which appointments are scheduled, as they can provide information on his availability.

4. **Visit Website/Profile**: If Jettro has a professional website or social media profile, he might have an appointment booking system or calendar visibility that you can use to find open times.

5. **Mutual Contacts**: Ask mutual contacts if they know when he is free and willing to make appointments.

Remember to respect privacy boundaries and ensure any cont

In [6]:
def run_question_with_system_msg(question):
    llm_response = generate(
        prompt=question,
        model=MODEL,
        options={
            "temperature": 1,
        },
        system=(
            "You are a scheduling assistant. You help in checking the availability of people."
            "Do not make up availability if you do not know the person's schedule."
            "Answer in short sentences, stick to the answer to the question."
        )
    )
    main_log.debug(llm_response)
    return llm_response.response


In [7]:
print(run_question_with_system_msg("When is Jettro available?"))

I don't have access to Jettro's schedule. Please check with Jettro directly or consult a shared calendar if one is used for scheduling.
